# 🏋️‍♀️ Web Scraping Practice Exercises

## Put Your Skills to the Test!

Now that you've learned the basics of `requests` and `BeautifulSoup`, it's time to practice.

**Target Website:** https://quotes.toscrape.com/

This website contains quotes from famous people and is designed specifically for scraping practice.

---

## 🛠️ Setup

Run the cell below to import the necessary libraries.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

## 🟢 Challenge 1: Scrape All Quotes on the First Page

**Goal:** Extract the **text**, **author**, and **tags** for every quote on the homepage.

**Instructions:**
1. Fetch the homepage (`https://quotes.toscrape.com/`)
2. Inspect the HTML to find the quote container
3. Extract the Quote Text, Author Name, and Tags (as a list)
4. Store them in a list of dictionaries

In [2]:
# Your Code Here
url = "https://quotes.toscrape.com/"

# 1. Fetch the page
response = requests.get(url)

# 2. Parse HTML
soup = BeautifulSoup(response.content, 'html.parser')

# 3. Find quote elements
quotes = soup.find_all('div', class_='quote')

# 4. Loop through and extract data
for quote in quotes:
    text = quote.find('span', class_='text').get_text()
    author = quote.find('small', class_='author').get_text()
    print(f"Quote: {text}\nAuthor: {author}")

# Hint: Look for <div class="quote">


Quote: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Author: Albert Einstein
Quote: “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Author: J.K. Rowling
Quote: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Author: Albert Einstein
Quote: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Author: Jane Austen
Quote: “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Author: Marilyn Monroe
Quote: “Try not to become a man of success. Rather become a man of value.”
Author: Albert Einstein
Quote: “It is better to be hated for what you are than to be loved for what you are not.”
Author: André Gide
Quote: “I have not failed. I've just found 10,000 ways that won't work.”
Author: Thomas A. Edi

In [3]:
# Solution Code (Try not to peek until you've tried!)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find_all('div', class_='quote')

data = []
for quote in quotes:
    text = quote.find('span', class_='text').get_text(strip=True)
    author = quote.find('small', class_='author').get_text(strip=True)
    tags = [tag.get_text(strip=True) for tag in quote.find_all('a', class_='tag')]
    data.append({'quote': text, 'author': author, 'tags': tags})

print(f"Found {len(data)} quotes:\n")
print(data[0])

Found 10 quotes:

{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}


## 🟡 Challenge 2: Scrape Author Details

**Goal:** Follow the "(about)" link for each author and scrape their **birth date** and **birth location**.

**Instructions:**
1. Use the data from Challenge 1
2. Find the "(about)" link next to the author's name
3. Fetch that new URL
4. Extract the biography details

*Note: This requires nested requests! Be careful not to hammer the server.*

In [6]:
# Assuming you have the 'quotes' list from Challenge 1
first_quote = quotes[0]  # Let's test with the first quote

# 1. Find the 'about' link (it's the first <a> tag inside the span with the author)
# We can search for the <a> tag directly inside the quote container
link_tag = first_quote.find('a') 
# Or be more specific: find the string "(about)"
# link_tag = first_quote.find('a', string='(about)')

# 2. Get the URL
relative_url = link_tag['href']  # e.g., "/author/Albert-Einstein"
full_url = "https://quotes.toscrape.com" + relative_url

print(f"Fetching author info from: {full_url}")

# 3. Request the author page
author_page = requests.get(full_url)
author_soup = BeautifulSoup(author_page.text, 'lxml')

# 4. Extract details
born_date = author_soup.find('span', class_='author-born-date').get_text()
born_loc = author_soup.find('span', class_='author-born-location').get_text()

print(f"Born: {born_date} {born_loc}")

Fetching author info from: https://quotes.toscrape.com/author/Albert-Einstein
Born: March 14, 1879 in Ulm, Germany


In [7]:
# Solution Code
first_quote = quotes[0]
author_link = first_quote.find('a', href=True)['href']  # Relative link like /author/Albert-Einstein
full_author_url = "https://quotes.toscrape.com" + author_link

print(f"Fetching author info from: {full_author_url}")
auth_response = requests.get(full_author_url)
auth_soup = BeautifulSoup(auth_response.text, 'lxml')

born_date = auth_soup.find('span', class_='author-born-date').get_text()
born_loc = auth_soup.find('span', class_='author-born-location').get_text()

print(f"Born: {born_date} {born_loc}")

Fetching author info from: https://quotes.toscrape.com/author/Albert-Einstein
Born: March 14, 1879 in Ulm, Germany


## 🔴 Challenge 3: Scrape All Pages (Pagination)

**Goal:** Write a script that automatically scrapes **ALL** quotes from the entire website, not just the first page.

**Instructions:**
1. Start at page 1
2. Look for the "Next" button (`<li class="next">`)
3. If it exists, get the URL and scrape that page
4. Repeat until there is no "Next" button

**Tip:** Use a `while` loop!

In [8]:
# Your Code Here

base_url = "https://quotes.toscrape.com"
current_url = base_url
all_quotes = []

# while True:
#     ...
while True:
    response = requests.get(current_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    quotes = soup.find_all('span', class_='text')
    for quote in quotes:
        all_quotes.append(quote.text)
    next_page = soup.find('li', class_='next')
    if not next_page:
        break
    current_url = base_url + next_page.find('a')['href']
    time.sleep(1)

print(all_quotes)

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", '“Try not to become a man of success. Rather become a man of value.”', '“It is better to be hated for what you are than to be loved for what you are not.”', "“I have not failed. I've just found 10,000 ways that won't work.”", "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”", '“A day without sunshine is like, you know, night.”', "“This life is what you make it. No matter what, you're going to

In [9]:
# Solution Code
url = "https://quotes.toscrape.com"
all_data = []

while True:
    print(f"Scraping: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    # Extract quotes
    quotes = soup.find_all('div', class_='quote')
    for quote in quotes:
        text = quote.find('span', class_='text').get_text()
        author = quote.find('small', class_='author').get_text()
        all_data.append({'quote': text, 'author': author})
        
    # Find next button
    next_btn = soup.find('li', class_='next')
    if next_btn:
        next_link = next_btn.find('a')['href']
        url = "https://quotes.toscrape.com" + next_link
        time.sleep(0.5)  # Be polite
    else:
        print("No more pages!")
        break

print(f"Total quotes collected: {len(all_data)}")

Scraping: https://quotes.toscrape.com
Scraping: https://quotes.toscrape.com/page/2/
Scraping: https://quotes.toscrape.com/page/3/
Scraping: https://quotes.toscrape.com/page/4/
Scraping: https://quotes.toscrape.com/page/5/
Scraping: https://quotes.toscrape.com/page/6/
Scraping: https://quotes.toscrape.com/page/7/
Scraping: https://quotes.toscrape.com/page/8/
Scraping: https://quotes.toscrape.com/page/9/
Scraping: https://quotes.toscrape.com/page/10/
No more pages!
Total quotes collected: 100


## 🏆 Bonus Challenge: Top 10 Tags

**Goal:** Isolate the list of top 10 tags shown on the right sidebar of the homepage.

**Instructions:**
1. Inspect the sidebar
2. Find the container for tags
3. Extract the text of the top 10 tags

In [ ]:
# Your Code Here


In [ ]:
# Solution Code
response = requests.get("https://quotes.toscrape.com")
soup = BeautifulSoup(response.text, 'lxml')

tag_box = soup.find('div', class_='tags-box')
top_tags = [tag.get_text(strip=True) for tag in tag_box.find_all('a', class_='tag')]

print("Top 10 Tags:")
print(top_tags)

## 🏁 Final Step: Export to Excel

Turn your scraped data into a nice Excel report.

In [10]:
df = pd.DataFrame(all_data)
df.to_excel("quotes_report.xlsx", index=False)
print("Saved to quotes_report.xlsx")
df.head()

Saved to quotes_report.xlsx


,quote,author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
